# Relatório de Análise - TCC

## Análise de avaliações de desempenho para o TCC

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline

In [3]:
pd.read_csv('C:/Users/kaiom/Documents/TCC/Dados/Atualizados/Cliente_1/report_notas_comp.csv', low_memory = False, sep=',')

ParserError: Error tokenizing data. C error: Expected 14 fields in line 88, saw 16


In [ ]:
dados = pd.read_csv('C:/Users/kaiom/Documents/TCC/Dados/Atualizados/Cliente_1/report_notas_comp.csv', low_memory = False, sep=',')

In [ ]:
dados[['Assunto CPF/Login']].drop_duplicates()

In [ ]:
competenciasValidas = dados[['Nome do modelo do formulário', 'Nome da Competência']]
competenciasValidas.head()

In [ ]:
competenciasValidas = competenciasValidas.drop_duplicates()
competenciasValidas

In [ ]:
competencias = pd.DataFrame(competenciasValidas.groupby('Nome da Competência').size(), columns=None).reset_index()


In [ ]:
competencias.columns = ['Competencia','Frequencia']
competencias

In [ ]:
competenciasTodosAnos = competencias[ competencias['Frequencia'] == 4 ]
competenciasTodosAnos

In [ ]:
listaCompValidas = list(competenciasTodosAnos['Competencia'])
listaCompValidas

In [ ]:
selecao = dados['Nome da Competência'].isin(listaCompValidas)

In [ ]:
dados_selecionados = dados[selecao]
dados_selecionados.columns = ['idModelo','nomeModelo','idDocumento','userId','cargoId','funcao','area','dtNascimento','dtDesligamento','dtAdmissao','gestorId','notaGlobal','competenciaId','nomeCompetencia','autoAv','gestorAv']


In [ ]:
participantesAno = dados_selecionados[['idModelo','userId']].drop_duplicates()
participantesAno = participantesAno.sort_values(by=['userId'])
participantesAno = pd.DataFrame(participantesAno.groupby('userId').size(), columns=None).reset_index()
participantesAno.columns = ['Usuario','Frequencia']
participantesValidos = participantesAno[ participantesAno['Frequencia'] == 4 ]
participantesValidos = participantesValidos.drop_duplicates()
listaParticipantesValidos = list(participantesValidos['Usuario'])
selecaoParticipantes = dados_selecionados['userId'].isin(listaParticipantesValidos)
dadosFinais = dados_selecionados[selecaoParticipantes]
dadosFinais

## Número de avaliados por ano

In [ ]:
infoAvaliacoes = dadosFinais[['nomeModelo','userId']].drop_duplicates()
infoAvaliacoes = pd.DataFrame(infoAvaliacoes.groupby('nomeModelo').size(), columns=None).reset_index()
infoAvaliacoes

In [ ]:
df = dadosFinais.pivot_table(index=["nomeModelo","userId","nomeCompetencia"], values="gestorAv")
df

## Join entre avaliações e PDI

In [ ]:
pdiMapeado = pd.read_csv('C:/Users/kaiom/Documents/TCC/Dados/Atualizados/Cliente_1/report_pdi.csv', low_memory = False, sep=',')
pdiMapeadoAjustado = pdiMapeado [['Nome do plano do Objetivo/meta de Desenvolvimento','Proprietário do Objetivo/meta CPF/Login','Nome da Competência','Nome da Competência']]
pdiMapeadoAjustado.columns = ["nomeModelo","userId","nomeCompetencia",'Competência']
pdiMapeadoAjustado = pdiMapeadoAjustado.set_index(["nomeModelo","userId","nomeCompetencia"])
pdiMapeadoAjustado.head()

In [ ]:
listaPdiMapeadoPorCompetencia = pd.merge(df.reset_index(), pdiMapeadoAjustado.reset_index(), on=["nomeModelo","userId","nomeCompetencia"], how ='left')
listaPdiMapeadoPorCompetencia.columns = [['Ano', 'userId','Competência','gestorAv','PDI?']]


In [ ]:
listaPdiMapeadoPorCompetencia['PDI?'] = listaPdiMapeadoPorCompetencia['PDI?'].replace(to_replace =['Atuação integrada e estratégica',
 'Autodesenvolvimento',
 'Comunicação',
 'Conhecimento do Mercado',
 'Domínio técnico',
 'Excelência Operacional',
 'Foco em resultado',
 'Foco no associado',
 'Gestão e desenvolvimento de Pessoas',
 'Identidade Institucional',
 'Melhoria contínua',
 'Promoção da Mudança',
 'Tomada de decisão',
 'Trabalho em equipe'], value ="Sim")

listaPdiMapeadoPorCompetencia['PDI?'] = listaPdiMapeadoPorCompetencia['PDI?'].fillna('Não')


In [ ]:
listaPdiMapeadoPorCompetencia.columns = ['anoAv','colaborador','comp','avaliacao','pdiAnoAnterior']

In [ ]:
listaPdiMapeadoPorCompetencia.pivot_table(index=["colaborador","comp"], columns=['anoAv'], values=["avaliacao"])


In [ ]:
data = listaPdiMapeadoPorCompetencia.pivot_table(index=["colaborador","comp"], columns=['anoAv'], values="avaliacao")

In [ ]:
data = pd.DataFrame(data.to_records())

In [ ]:
data["2016_analisado"] = np.where(data['2015'] < data['2016'], 'Maior', np.where(data['2015'] > data['2016'], 'Menor', np.where(data['2015'] == data['2016'], 'Igual','Não Mapeado')))
data["2017_analisado"] = np.where(data['2016'] < data['2017'], 'Maior', np.where(data['2016'] > data['2017'], 'Menor', np.where(data['2016'] == data['2017'], 'Igual','Não Mapeado')))
data["2018_analisado"] = np.where(data['2017'] < data['2018'], 'Maior', np.where(data['2017'] > data['2018'], 'Menor', np.where(data['2017'] == data['2018'], 'Igual','Não Mapeado')))

In [ ]:
analiseDados = data[['colaborador','comp','2016_analisado','2017_analisado','2018_analisado']]
analiseDados.set_index(['colaborador','comp'])

In [ ]:
analiseCompetenciaEntreAnos = analiseDados.melt(id_vars=['colaborador','comp'],value_vars=['2016_analisado','2017_analisado','2018_analisado'])
analiseCompetenciaEntreAnos.columns = ['colaborador','comp','anoAv','resultado']

analiseCompetenciaEntreAnos["anoAv"] = np.where(analiseCompetenciaEntreAnos["anoAv"] == "2016_analisado", '2016', np.where(analiseCompetenciaEntreAnos["anoAv"] == "2017_analisado", '2017', np.where(analiseCompetenciaEntreAnos["anoAv"] == "2018_analisado", '2018', '')))
analiseCompetenciaEntreAnos['anoAv'] = analiseCompetenciaEntreAnos.anoAv.astype(str)
listaPdiMapeadoPorCompetencia['anoAv'] = listaPdiMapeadoPorCompetencia.anoAv.astype(str)

## Tabela final de Análise - PDI e Análise entre anos

In [ ]:
tabelaFinal = pd.merge(analiseCompetenciaEntreAnos,listaPdiMapeadoPorCompetencia,how='left',on=['colaborador','comp','anoAv'])


In [ ]:
tabelaFinal['pdiAnoAnterior'] = tabelaFinal['pdiAnoAnterior'].fillna('Não')
resultadosNaoMapeados = tabelaFinal
tabelaFinal = tabelaFinal[tabelaFinal.resultado != 'Não Mapeado']

## Resultados incluindo competências "Não Mapeadas"

In [ ]:
comPdi = tabelaFinal['pdiAnoAnterior'] == 'Sim'
semPdi = tabelaFinal['pdiAnoAnterior'] == 'Não'
resultadosComPdi = tabelaFinal[comPdi]
resultadosSemPdi = tabelaFinal[semPdi]

comPdi_2016 = resultadosComPdi['anoAv'] == '2016'
semPdi_2016 = resultadosSemPdi['anoAv'] == '2016'
resultadosComPdi_2016 = resultadosComPdi[comPdi_2016]
resultadosSemPdi_2016 = resultadosSemPdi[semPdi_2016]

comPdi_2017 = resultadosComPdi['anoAv'] == '2017'
semPdi_2017 = resultadosSemPdi['anoAv'] == '2017'
resultadosComPdi_2017 = resultadosComPdi[comPdi_2017]
resultadosSemPdi_2017 = resultadosSemPdi[semPdi_2017]

comPdi_2018 = resultadosComPdi['anoAv'] == '2018'
semPdi_2018 = resultadosSemPdi['anoAv'] == '2018'
resultadosComPdi_2018 = resultadosComPdi[comPdi_2018]
resultadosSemPdi_2018 = resultadosSemPdi[semPdi_2018]

## Resultados Gerais

In [ ]:
resultadosComPdi['resultado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'], title='Resultado das competências que tiveram o PDI mapeado no ano anterior')
plt.savefig('pdiMapeadoAnoAnterior.png', bbox_inches='tight')

In [ ]:
resultadosSemPdi['resultado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'], title='Resultado das competências não que tiveram o PDI mapeado no ano anterior')
plt.savefig('pdiNaoMapeadoAnoAnterior.png', bbox_inches='tight')

## [2015 x 2016] Resultados

In [ ]:
analiseDados['2016_analisado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'], title='Análise geral da avaliação de competências 2015 x 2016')
plt.savefig('avComp2015-2016.png', bbox_inches='tight')

In [ ]:
resultadosComPdi_2016['resultado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'], title='[2016] Resultado das competências que tiveram o PDI mapeado no ano anterior')
plt.savefig('avCompPdiMapeado2015-2016.png', bbox_inches='tight')

In [ ]:
resultadosSemPdi_2016['resultado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'], title='[2016] Resultado das competências que NÃO tiveram o PDI mapeado no ano anterior')
plt.savefig('avCompPdiNaoMapeado2015-2016.png', bbox_inches='tight')

## [2016 x 2017] Resultados

In [ ]:
analiseDados['2017_analisado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'],title='Análise geral da avaliação de competências 2016 x 2017')
plt.savefig('avComp2016-2017.png', bbox_inches='tight')

In [ ]:
resultadosComPdi_2017['resultado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'], title='[2017] Resultado das competências que tiveram o PDI mapeado no ano anterior')
plt.savefig('avCompPdiMapeado2016-2017.png', bbox_inches='tight')

In [ ]:
resultadosSemPdi_2017['resultado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'], title='[2017] Resultado das competências que NÃO tiveram o PDI mapeado no ano anterior')
plt.savefig('avCompPdiNaoMapeado2016-2017.png', bbox_inches='tight')

## [2017 x 2018] Resultados

In [ ]:
analiseDados['2018_analisado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'],title='Análise geral da avaliação de competências 2017 x 2018')
plt.savefig('avComp2017-2018.png', bbox_inches='tight')

In [ ]:
resultadosComPdi_2018['resultado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'],title='[2018] Resultado das competências que tiveram o PDI mapeado no ano anterior')
plt.savefig('avCompPdiMapeado2017-2018.png', bbox_inches='tight')

In [ ]:
resultadosSemPdi_2018['resultado'].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#b0efff', '#d0ffb0', '#ffb4b0'], title='[2018] Resultado das competências que NÃO tiveram o PDI mapeado no ano anterior')
plt.savefig('avCompPdiNaoMapeado2017-2018.png', bbox_inches='tight')

## Gráfico de  barras empilhadas da avaliação de competência ao longo dos processos

# [2015x2016] Com PDI

In [ ]:
mapeados_2016 = resultadosComPdi_2016['resultado'] != 'Não Mapeado'
resultadoComPdi_2016_excluindoNaoMapeados = resultadosComPdi_2016[mapeados_2016]
resultadoComPdi_2016_excluindoNaoMapeados
resultadoComPdi_2016_excluindoNaoMapeados_analisado = resultadoComPdi_2016_excluindoNaoMapeados.pivot_table(index=['comp'],columns=['resultado'], values=['anoAv'],aggfunc='count')
resultadoComPdi_2016_excluindoNaoMapeados_analisado = pd.DataFrame(resultadoComPdi_2016_excluindoNaoMapeados_analisado.to_records())
resultadoComPdi_2016_excluindoNaoMapeados_analisado.columns = ['comp','Igual','Maior','Menor']
df = resultadoComPdi_2016_excluindoNaoMapeados_analisado
df

In [ ]:
# Create a figure with a single subplot
f, ax = plt.subplots(1, figsize=(10,5))
colunas = ['Igual','Maior','Menor']
# Set bar width at 1
bar_width = 1

# positions of the left bar-boundaries
bar_l = [i for i in range(len(df['Igual']))] 

# positions of the x-axis ticks (center of the bars as bar labels)
tick_pos = [i+(bar_width/2) for i in bar_l] 
tick_pos_div = [i for i in bar_l] 

# Create the total score for each participant
totals = [i+j+k for i,j,k in zip(df['Igual'], df['Maior'], df['Menor'])]

# Create the percentage of the total score the pre_score value for each participant was
pre_rel = [i / j * 100 for  i,j in zip(df['Igual'], totals)]
pre_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Igual'], totals)]

# Create the percentage of the total score the mid_score value for each participant was
mid_rel = [i / j * 100 for  i,j in zip(df['Maior'], totals)]
mid_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Maior'], totals)] 
mid_rel_pos2 = [i + j for  i,j in zip(pre_rel, mid_rel_pos)] 

# Create the percentage of the total score the post_score value for each participant was
post_rel = [i / j * 100 for  i,j in zip(df['Menor'], totals)]
post_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Menor'], totals)]
post_rel_pos2 = [i + j + y for  i,j,y in zip(pre_rel, mid_rel,post_rel_pos)]

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using pre_rel data
       pre_rel, 
       # labeled 
       label='Igual', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#b0efff',
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, pre_rel_pos, pre_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using mid_rel data
       mid_rel, 
       # with pre_rel
       bottom=pre_rel, 
       # labeled 
       label='Maior', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#d0ffb0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )

for xpos, ypos, yval in zip(tick_pos_div, mid_rel_pos2, mid_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using post_rel data
       post_rel, 
       # with pre_rel and mid_rel on bottom
       bottom=[i+j for i,j in zip(pre_rel, mid_rel)], 
       # labeled 
       label='Menor',
       # with alpha
       alpha=0.9, 
       # with color
       color='#ffb4b0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, post_rel_pos2, post_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Set the ticks to be first names
plt.xticks(tick_pos, df['comp'])
ax.set_ylabel("Percentual")
ax.set_xlabel("")

# Let the borders of the graphic
plt.xlim([min(tick_pos)-bar_width, max(tick_pos)+bar_width])
plt.ylim(-10, 110)

# rotate axis labels
plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')


# shot plot
plt.title('[2015x2016] Competências com PDI mapeado no ano anterior')
plt.legend(labels = colunas,fontsize= 14, loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()
plt.savefig('barrasEmpilhadas - ComPDI2015-2016.png', bbox_inches='tight')


# [2015x2016] Sem PDI

In [ ]:
mapeados_2016 = resultadosSemPdi_2016['resultado'] != 'Não Mapeado'
resultadoSemPdi_2016_excluindoNaoMapeados = resultadosSemPdi_2016[mapeados_2016]
resultadoSemPdi_2016_excluindoNaoMapeados
resultadoSemPdi_2016_excluindoNaoMapeados_analisado = resultadoSemPdi_2016_excluindoNaoMapeados.pivot_table(index=['comp'],columns=['resultado'], values=['anoAv'],aggfunc='count')
resultadoSemPdi_2016_excluindoNaoMapeados_analisado = pd.DataFrame(resultadoSemPdi_2016_excluindoNaoMapeados_analisado.to_records())
resultadoSemPdi_2016_excluindoNaoMapeados_analisado.columns = ['comp','Igual','Maior','Menor']
df = resultadoSemPdi_2016_excluindoNaoMapeados_analisado
df

In [ ]:
# Create a figure with a single subplot
f, ax = plt.subplots(1, figsize=(10,5))
colunas = ['Igual','Maior','Menor']
# Set bar width at 1
bar_width = 1

# positions of the left bar-boundaries
bar_l = [i for i in range(len(df['Igual']))] 

# positions of the x-axis ticks (center of the bars as bar labels)
tick_pos = [i+(bar_width/2) for i in bar_l] 
tick_pos_div = [i for i in bar_l] 

# Create the total score for each participant
totals = [i+j+k for i,j,k in zip(df['Igual'], df['Maior'], df['Menor'])]

# Create the percentage of the total score the pre_score value for each participant was
pre_rel = [i / j * 100 for  i,j in zip(df['Igual'], totals)]
pre_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Igual'], totals)]

# Create the percentage of the total score the mid_score value for each participant was
mid_rel = [i / j * 100 for  i,j in zip(df['Maior'], totals)]
mid_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Maior'], totals)] 
mid_rel_pos2 = [i + j for  i,j in zip(pre_rel, mid_rel_pos)] 

# Create the percentage of the total score the post_score value for each participant was
post_rel = [i / j * 100 for  i,j in zip(df['Menor'], totals)]
post_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Menor'], totals)]
post_rel_pos2 = [i + j + y for  i,j,y in zip(pre_rel, mid_rel,post_rel_pos)]

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using pre_rel data
       pre_rel, 
       # labeled 
       label='Igual', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#b0efff',
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, pre_rel_pos, pre_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using mid_rel data
       mid_rel, 
       # with pre_rel
       bottom=pre_rel, 
       # labeled 
       label='Maior', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#d0ffb0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )

for xpos, ypos, yval in zip(tick_pos_div, mid_rel_pos2, mid_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using post_rel data
       post_rel, 
       # with pre_rel and mid_rel on bottom
       bottom=[i+j for i,j in zip(pre_rel, mid_rel)], 
       # labeled 
       label='Menor',
       # with alpha
       alpha=0.9, 
       # with color
       color='#ffb4b0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, post_rel_pos2, post_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Set the ticks to be first names
plt.xticks(tick_pos, df['comp'])
ax.set_ylabel("Percentual")
ax.set_xlabel("")

# Let the borders of the graphic
plt.xlim([min(tick_pos)-bar_width, max(tick_pos)+bar_width])
plt.ylim(-10, 110)

# rotate axis labels
plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')


# shot plot
plt.title('[2015x2016] Competências sem PDI mapeado no ano anterior')
plt.legend(labels = colunas,fontsize= 14, loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()
plt.savefig('barrasEmpilhadas - SemPDI2015-2016.png', bbox_inches='tight')

# [2016x2017] Com PDI

In [ ]:
mapeados_2017 = resultadosComPdi_2017['resultado'] != 'Não Mapeado'
resultadoComPdi_2017_excluindoNaoMapeados = resultadosComPdi_2017[mapeados_2017]
resultadoComPdi_2017_excluindoNaoMapeados
resultadoComPdi_2017_excluindoNaoMapeados_analisado = resultadoComPdi_2017_excluindoNaoMapeados.pivot_table(index=['comp'],columns=['resultado'], values=['anoAv'],aggfunc='count')
resultadoComPdi_2017_excluindoNaoMapeados_analisado = pd.DataFrame(resultadoComPdi_2017_excluindoNaoMapeados_analisado.to_records())
resultadoComPdi_2017_excluindoNaoMapeados_analisado.columns = ['comp','Igual','Maior','Menor']
df = resultadoComPdi_2017_excluindoNaoMapeados_analisado
df

In [ ]:
# Create a figure with a single subplot
f, ax = plt.subplots(1, figsize=(10,5))
colunas = ['Igual','Maior','Menor']
# Set bar width at 1
bar_width = 1

# positions of the left bar-boundaries
bar_l = [i for i in range(len(df['Igual']))] 

# positions of the x-axis ticks (center of the bars as bar labels)
tick_pos = [i+(bar_width/2) for i in bar_l] 
tick_pos_div = [i for i in bar_l] 

# Create the total score for each participant
totals = [i+j+k for i,j,k in zip(df['Igual'], df['Maior'], df['Menor'])]

# Create the percentage of the total score the pre_score value for each participant was
pre_rel = [i / j * 100 for  i,j in zip(df['Igual'], totals)]
pre_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Igual'], totals)]

# Create the percentage of the total score the mid_score value for each participant was
mid_rel = [i / j * 100 for  i,j in zip(df['Maior'], totals)]
mid_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Maior'], totals)] 
mid_rel_pos2 = [i + j for  i,j in zip(pre_rel, mid_rel_pos)] 

# Create the percentage of the total score the post_score value for each participant was
post_rel = [i / j * 100 for  i,j in zip(df['Menor'], totals)]
post_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Menor'], totals)]
post_rel_pos2 = [i + j + y for  i,j,y in zip(pre_rel, mid_rel,post_rel_pos)]

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using pre_rel data
       pre_rel, 
       # labeled 
       label='Igual', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#b0efff',
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, pre_rel_pos, pre_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using mid_rel data
       mid_rel, 
       # with pre_rel
       bottom=pre_rel, 
       # labeled 
       label='Maior', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#d0ffb0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )

for xpos, ypos, yval in zip(tick_pos_div, mid_rel_pos2, mid_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using post_rel data
       post_rel, 
       # with pre_rel and mid_rel on bottom
       bottom=[i+j for i,j in zip(pre_rel, mid_rel)], 
       # labeled 
       label='Menor',
       # with alpha
       alpha=0.9, 
       # with color
       color='#ffb4b0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, post_rel_pos2, post_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Set the ticks to be first names
plt.xticks(tick_pos, df['comp'])
ax.set_ylabel("Percentual")
ax.set_xlabel("")

# Let the borders of the graphic
plt.xlim([min(tick_pos)-bar_width, max(tick_pos)+bar_width])
plt.ylim(-10, 110)

# rotate axis labels
plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')


# shot plot
plt.title('[2016x2017] Competências com PDI mapeado no ano anterior')
plt.legend(labels = colunas,fontsize= 14, loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()
plt.savefig('barrasEmpilhadas - ComPDI2016-2017.png', bbox_inches='tight')

# [2016x2017] Sem PDI

In [ ]:
mapeados_2017 = resultadosSemPdi_2017['resultado'] != 'Não Mapeado'
resultadoSemPdi_2017_excluindoNaoMapeados = resultadosSemPdi_2017[mapeados_2017]
resultadoSemPdi_2017_excluindoNaoMapeados
resultadoSemPdi_2017_excluindoNaoMapeados_analisado = resultadoSemPdi_2017_excluindoNaoMapeados.pivot_table(index=['comp'],columns=['resultado'], values=['anoAv'],aggfunc='count')
resultadoSemPdi_2017_excluindoNaoMapeados_analisado = pd.DataFrame(resultadoSemPdi_2017_excluindoNaoMapeados_analisado.to_records())
resultadoSemPdi_2017_excluindoNaoMapeados_analisado.columns = ['comp','Igual','Maior','Menor']
df = resultadoSemPdi_2017_excluindoNaoMapeados_analisado
df

In [ ]:
# Create a figure with a single subplot
f, ax = plt.subplots(1, figsize=(10,5))
colunas = ['Igual','Maior','Menor']
# Set bar width at 1
bar_width = 1

# positions of the left bar-boundaries
bar_l = [i for i in range(len(df['Igual']))] 

# positions of the x-axis ticks (center of the bars as bar labels)
tick_pos = [i+(bar_width/2) for i in bar_l] 
tick_pos_div = [i for i in bar_l] 

# Create the total score for each participant
totals = [i+j+k for i,j,k in zip(df['Igual'], df['Maior'], df['Menor'])]

# Create the percentage of the total score the pre_score value for each participant was
pre_rel = [i / j * 100 for  i,j in zip(df['Igual'], totals)]
pre_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Igual'], totals)]

# Create the percentage of the total score the mid_score value for each participant was
mid_rel = [i / j * 100 for  i,j in zip(df['Maior'], totals)]
mid_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Maior'], totals)] 
mid_rel_pos2 = [i + j for  i,j in zip(pre_rel, mid_rel_pos)] 

# Create the percentage of the total score the post_score value for each participant was
post_rel = [i / j * 100 for  i,j in zip(df['Menor'], totals)]
post_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Menor'], totals)]
post_rel_pos2 = [i + j + y for  i,j,y in zip(pre_rel, mid_rel,post_rel_pos)]

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using pre_rel data
       pre_rel, 
       # labeled 
       label='Igual', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#b0efff',
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, pre_rel_pos, pre_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using mid_rel data
       mid_rel, 
       # with pre_rel
       bottom=pre_rel, 
       # labeled 
       label='Maior', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#d0ffb0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )

for xpos, ypos, yval in zip(tick_pos_div, mid_rel_pos2, mid_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using post_rel data
       post_rel, 
       # with pre_rel and mid_rel on bottom
       bottom=[i+j for i,j in zip(pre_rel, mid_rel)], 
       # labeled 
       label='Menor',
       # with alpha
       alpha=0.9, 
       # with color
       color='#ffb4b0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, post_rel_pos2, post_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Set the ticks to be first names
plt.xticks(tick_pos, df['comp'])
ax.set_ylabel("Percentual")
ax.set_xlabel("")

# Let the borders of the graphic
plt.xlim([min(tick_pos)-bar_width, max(tick_pos)+bar_width])
plt.ylim(-10, 110)

# rotate axis labels
plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')


# shot plot
plt.title('[2016x2017] Competências sem PDI mapeado no ano anterior')
plt.legend(labels = colunas,fontsize= 14, loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()
plt.savefig('barrasEmpilhadas - SemPDI2016-2017.png', bbox_inches='tight')

# [2017x2018] Com PDI

In [ ]:
mapeados_2018 = resultadosComPdi_2018['resultado'] != 'Não Mapeado'
resultadoComPdi_2018_excluindoNaoMapeados = resultadosComPdi_2018[mapeados_2018]
resultadoComPdi_2018_excluindoNaoMapeados
resultadoComPdi_2018_excluindoNaoMapeados_analisado = resultadoComPdi_2018_excluindoNaoMapeados.pivot_table(index=['comp'],columns=['resultado'], values=['anoAv'],aggfunc='count')
resultadoComPdi_2018_excluindoNaoMapeados_analisado = pd.DataFrame(resultadoComPdi_2018_excluindoNaoMapeados_analisado.to_records())
resultadoComPdi_2018_excluindoNaoMapeados_analisado.columns = ['comp','Igual','Maior','Menor']
df = resultadoComPdi_2018_excluindoNaoMapeados_analisado

df['Maior'] = df['Maior'].fillna(value=0)
df

In [ ]:
# Create a figure with a single subplot
f, ax = plt.subplots(1, figsize=(10,5))
colunas = ['Igual','Maior','Menor']
# Set bar width at 1
bar_width = 1

# positions of the left bar-boundaries
bar_l = [i for i in range(len(df['Igual']))] 

# positions of the x-axis ticks (center of the bars as bar labels)
tick_pos = [i+(bar_width/2) for i in bar_l] 
tick_pos_div = [i for i in bar_l] 

# Create the total score for each participant
totals = [i+j+k for i,j,k in zip(df['Igual'], df['Maior'], df['Menor'])]

# Create the percentage of the total score the pre_score value for each participant was
pre_rel = [i / j * 100 for  i,j in zip(df['Igual'], totals)]
pre_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Igual'], totals)]

# Create the percentage of the total score the mid_score value for each participant was
mid_rel = [i / j * 100 for  i,j in zip(df['Maior'], totals)]
mid_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Maior'], totals)] 
mid_rel_pos2 = [i + j for  i,j in zip(pre_rel, mid_rel_pos)] 

# Create the percentage of the total score the post_score value for each participant was
post_rel = [i / j * 100 for  i,j in zip(df['Menor'], totals)]
post_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Menor'], totals)]
post_rel_pos2 = [i + j + y for  i,j,y in zip(pre_rel, mid_rel,post_rel_pos)]

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using pre_rel data
       pre_rel, 
       # labeled 
       label='Igual', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#b0efff',
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, pre_rel_pos, pre_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using mid_rel data
       mid_rel, 
       # with pre_rel
       bottom=pre_rel, 
       # labeled 
       label='Maior', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#d0ffb0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )

for xpos, ypos, yval in zip(tick_pos_div, mid_rel_pos2, mid_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using post_rel data
       post_rel, 
       # with pre_rel and mid_rel on bottom
       bottom=[i+j for i,j in zip(pre_rel, mid_rel)], 
       # labeled 
       label='Menor',
       # with alpha
       alpha=0.9, 
       # with color
       color='#ffb4b0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, post_rel_pos2, post_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Set the ticks to be first names
plt.xticks(tick_pos, df['comp'])
ax.set_ylabel("Percentual")
ax.set_xlabel("")

# Let the borders of the graphic
plt.xlim([min(tick_pos)-bar_width, max(tick_pos)+bar_width])
plt.ylim(-10, 110)

# rotate axis labels
plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')


# shot plot
plt.title('[2017x2018] Competências com PDI mapeado no ano anterior')
plt.legend(labels = colunas,fontsize= 14, loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()
plt.savefig('barrasEmpilhadas - ComPDI2017-2018.png', bbox_inches='tight')

# [2017x2018] Sem PDI

In [ ]:
mapeados_2018 = resultadosSemPdi_2018['resultado'] != 'Não Mapeado'
resultadoSemPdi_2018_excluindoNaoMapeados = resultadosSemPdi_2018[mapeados_2018]
resultadoSemPdi_2018_excluindoNaoMapeados
resultadoSemPdi_2018_excluindoNaoMapeados_analisado = resultadoSemPdi_2018_excluindoNaoMapeados.pivot_table(index=['comp'],columns=['resultado'], values=['anoAv'],aggfunc='count')
resultadoSemPdi_2018_excluindoNaoMapeados_analisado = pd.DataFrame(resultadoSemPdi_2018_excluindoNaoMapeados_analisado.to_records())
resultadoSemPdi_2018_excluindoNaoMapeados_analisado.columns = ['comp','Igual','Maior','Menor']
df = resultadoSemPdi_2018_excluindoNaoMapeados_analisado
df

In [ ]:
# Create a figure with a single subplot
f, ax = plt.subplots(1, figsize=(10,5))
colunas = ['Igual','Maior','Menor']
# Set bar width at 1
bar_width = 1

# positions of the left bar-boundaries
bar_l = [i for i in range(len(df['Igual']))] 

# positions of the x-axis ticks (center of the bars as bar labels)
tick_pos = [i+(bar_width/2) for i in bar_l] 
tick_pos_div = [i for i in bar_l] 

# Create the total score for each participant
totals = [i+j+k for i,j,k in zip(df['Igual'], df['Maior'], df['Menor'])]

# Create the percentage of the total score the pre_score value for each participant was
pre_rel = [i / j * 100 for  i,j in zip(df['Igual'], totals)]
pre_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Igual'], totals)]

# Create the percentage of the total score the mid_score value for each participant was
mid_rel = [i / j * 100 for  i,j in zip(df['Maior'], totals)]
mid_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Maior'], totals)] 
mid_rel_pos2 = [i + j for  i,j in zip(pre_rel, mid_rel_pos)] 

# Create the percentage of the total score the post_score value for each participant was
post_rel = [i / j * 100 for  i,j in zip(df['Menor'], totals)]
post_rel_pos = [(i / j * 100)/2 for  i,j in zip(df['Menor'], totals)]
post_rel_pos2 = [i + j + y for  i,j,y in zip(pre_rel, mid_rel,post_rel_pos)]

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using pre_rel data
       pre_rel, 
       # labeled 
       label='Igual', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#b0efff',
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, pre_rel_pos, pre_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using mid_rel data
       mid_rel, 
       # with pre_rel
       bottom=pre_rel, 
       # labeled 
       label='Maior', 
       # with alpha
       alpha=0.9, 
       # with color
       color='#d0ffb0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )

for xpos, ypos, yval in zip(tick_pos_div, mid_rel_pos2, mid_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Create a bar chart in position bar_1
ax.bar(bar_l, 
       # using post_rel data
       post_rel, 
       # with pre_rel and mid_rel on bottom
       bottom=[i+j for i,j in zip(pre_rel, mid_rel)], 
       # labeled 
       label='Menor',
       # with alpha
       alpha=0.9, 
       # with color
       color='#ffb4b0', 
       # with bar width
       width=bar_width,
       # with border color
       edgecolor='white'
       )
for xpos, ypos, yval in zip(tick_pos_div, post_rel_pos2, post_rel):
    plt.text(xpos, ypos, "%.1f"%yval + '%', ha="center", va="center",color='#000000')

# Set the ticks to be first names
plt.xticks(tick_pos, df['comp'])
ax.set_ylabel("Percentual")
ax.set_xlabel("")

# Let the borders of the graphic
plt.xlim([min(tick_pos)-bar_width, max(tick_pos)+bar_width])
plt.ylim(-10, 110)

# rotate axis labels
plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')


# shot plot
plt.title('[2017x2018] Competências sem PDI mapeado no ano anterior')
plt.legend(labels = colunas,fontsize= 14, loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()
plt.savefig('barrasEmpilhadas - SemPDI2017-2018.png')

## Análise não mapeados

In [ ]:
naoMapeadosComPdi = resultadosNaoMapeados['pdiAnoAnterior'] == 'Sim'
naoMapeadosSemPdi = resultadosNaoMapeados['pdiAnoAnterior'] == 'Não'
resultadosNaoMapeadosComPdi = resultadosNaoMapeados[naoMapeadosComPdi]
resultadosNaoMapeadosSemPdi = resultadosNaoMapeados[naoMapeadosSemPdi]

naoMapeados_2016 = resultadosNaoMapeados['anoAv'] == '2016'
resultadosNaoMapeados_2016 = resultadosNaoMapeados[naoMapeados_2016]

naoMapeados_2017 = resultadosNaoMapeados['anoAv'] == '2017'
resultadosNaoMapeados_2017 = resultadosNaoMapeados[naoMapeados_2017]

naoMapeados_2018 = resultadosNaoMapeados['anoAv'] == '2018'
resultadosNaoMapeados_2018 = resultadosNaoMapeados[naoMapeados_2018]

In [ ]:
resultadosNaoMapeados_2016["resultado"] = np.where(resultadosNaoMapeados_2016['resultado'] != 'Não Mapeado', 'Mapeado','Não Mapeado')

resultadosNaoMapeados_2017["resultado"] = np.where(resultadosNaoMapeados_2017['resultado'] != 'Não Mapeado', 'Mapeado','Não Mapeado')

resultadosNaoMapeados_2018["resultado"] = np.where(resultadosNaoMapeados_2018['resultado'] != 'Não Mapeado', 'Mapeado','Não Mapeado')


In [ ]:
resultadosNaoMapeados_2016["resultado"].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#d0ffb0', '#f77676'], title='[2015 x 2016] Análise de competências mapeadas ').set_ylabel('')

In [ ]:
resultadosNaoMapeados_2017["resultado"].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#d0ffb0', '#f77676'], title='[2016 x 2017] Análise de competências mapeadas ').set_ylabel('')


In [ ]:
resultadosNaoMapeados_2018["resultado"].value_counts().plot(kind='pie',autopct='%1.1f%%',figsize=(5,5),colors=['#d0ffb0', '#f77676'], title='[2017 x 2018] Análise de competências mapeadas ').set_ylabel('')
